# CSILSP-FAL

In [1]:
import gurobipy as gb
from CSILSPGenerator import Generator

#### Decision variables:

$$
Y_t = \begin{cases}
1 \text{ if } Z_{tk} > 0\\
0 \text{ otherwise }
\end{cases}
$$

$$
Z_{kt} = \begin{cases}
\text{ quantity produced/supplied
in period k } \\ \text{ to satisfy the demand in period t }
\end{cases}
$$

#### Parameters:

$$
T = \begin{cases}
\text{ number of periods in the horizon }
\end{cases}
$$

$$
d_t = \begin{cases}
\text{ Customer aggregate demand that must } \\ \text{ be delivered at the end of period t }
\end{cases}
$$

$$
h_t = \begin{cases}
\text{ Holding cost per unit of product in stock at the end of period t }
\end{cases}
$$

$$
p_t = \begin{cases}
\text{ Unit production/supplying cost in period t }
\end{cases}
$$

$$
s_t = \begin{cases}
\text{ Fixed setup cost in period t }
\end{cases}
$$

$$
C_t = \begin{cases}
\text{ Production/Supply capacity at period t }
\end{cases}
$$

#### CSILSP-FAL (disaggregate model)

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T}\sum_{k=t}^{T} (p_t + \sum_{j=t}^{k-1}h_j)Z_{tk} + \sum_{t=1}^{T}s_tY_t &\\
\text{s.t.} \;\;\;\;\;&\\
\sum_{k=1}^{t}Z_{kt} &= d_t \;\; , & \forall t \;\;\\
Z_{tk} &\leq d_kY_t \;\; , & \forall t,k \geq t \;\;\\
\sum_{k=t}^{T}Z_{tk} &\leq C_t \;\; , & \forall t \;\;\\
Z_{tk} &\geq 0 \;\; , & \forall t,k \geq t \;\;\\
Y_t &\in \{0,1\} \;\; , & \forall t \;\;
\end{alignat}
$$

In [2]:
# generation parameters

if __name__ == '__main__':
    T = 8
    l_b = 10
    u_b = 20

In [3]:
# generate instance

if __name__ == '__main__' or __name__ == '__autoexec__':
    instance = Generator(T)
    instance.generate(a=l_b,b=u_b)

In [4]:
if __name__ == '__main__':
    print(instance.printData())

    d   C   h   p   s
T                    
0  18  30  18  17  19
1  18  19  11  19  20
2  13  18  20  17  15
3  20  32  20  16  15
4  14  26  14  15  17
5  15  18  20  10  17
6  18  32  20  16  17
7  15  27  20  13  20


In [5]:
# variable bindings

T = instance.get_T()
d = instance.get_d()
C = instance.get_C()
h = instance.get_h()
p = instance.get_p()
s = instance.get_s()

In [6]:
# model

csilsp_fal = gb.Model()

Academic license - for non-commercial use only


In [7]:
# decision variables

Y = csilsp_fal.addVars(T,vtype=gb.GRB.BINARY,name='Y')

Z = csilsp_fal.addVars(T,T,vtype=gb.GRB.CONTINUOUS,name='Z')

csilsp_fal.update()
csilsp_fal.write('csilsp_fal.lp')

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T}\sum_{k=t}^{T} (p_t + \sum_{j=t}^{k-1}h_j)Z_{tk} + \sum_{t=1}^{T}s_tY_t &
\end{alignat}
$$

In [8]:
# objective function

objexpr = gb.quicksum( ( p[t] + (gb.quicksum( h[j] for j in range(t,k) )) )*Z[t,k] for t in range(T) for k in range(t,T) ) \
          + Y.prod(s)

csilsp_fal.setObjective(objexpr,gb.GRB.MINIMIZE)

csilsp_fal.update()
csilsp_fal.write('csilsp_fal.lp')

$$
\begin{alignat}{3}
\sum_{k=1}^{t}Z_{kt} &= d_t \;\; , & \forall t \;\;
\end{alignat}
$$

In [9]:
# constraints

csilsp_fal.addConstrs(( (gb.quicksum(Z[k,t] for k in range(t+1))) == d[t] for t in range(T) ),name='constrZ')

csilsp_fal.update()
csilsp_fal.write('csilsp_fal.lp')

$$
\begin{alignat}{3}
Z_{tk} &\leq d_kY_t \;\; , & \forall t,k \geq t \;\;
\end{alignat}
$$

In [10]:
csilsp_fal.addConstrs(( Z[t,k] <= d[k]*Y[t] for t in range(T) for k in range(t,T) ),name='constrZ2')

csilsp_fal.update()
csilsp_fal.write('csilsp_fal.lp')

$$
\begin{alignat}{3}
\sum_{k=t}^{T}Z_{tk} &\leq C_t \;\; , & \forall t \;\;
\end{alignat}
$$

In [11]:
csilsp_fal.addConstrs(( (gb.quicksum(Z[t,k] for k in range(t,T))) <= C[t] for t in range(T) ),name='constrZ3')

csilsp_fal.update()
csilsp_fal.write('csilsp_fal.lp')

$$
\begin{alignat}{3}
Z_{tk} &\geq 0 \;\; , & \forall t,k \geq t \;\;\\
\end{alignat}
$$

In [12]:
for t in range(T):
    for k in range(t,T):
        Z[t,k].lb = 0.0

csilsp_fal.update()
csilsp_fal.write('csilsp_fal.lp')

In [13]:
# solve model

csilsp_fal.optimize()

Optimize a model with 52 rows, 72 columns and 144 nonzeros
Variable types: 64 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 21 rows and 32 columns
Presolve time: 0.00s
Presolved: 31 rows, 40 columns, 108 nonzeros
Variable types: 34 continuous, 6 integer (6 binary)

Root relaxation: objective 2.172000e+03, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2172.0000000 2172.00000  0.00%     -    0s

Explored 0 nodes (6 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 2172 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.172000000000e+03, best bound 2.172000000000e+03, gap 0.0000%


In [14]:
if __name__ == '__main__':
    print("Soluzione ottima:",csilsp_fal.ObjVal)

Soluzione ottima: 2172.0


In [15]:
Y

{0: <gurobi.Var Y[0] (value 1.0)>,
 1: <gurobi.Var Y[1] (value 1.0)>,
 2: <gurobi.Var Y[2] (value 1.0)>,
 3: <gurobi.Var Y[3] (value 1.0)>,
 4: <gurobi.Var Y[4] (value 1.0)>,
 5: <gurobi.Var Y[5] (value 1.0)>,
 6: <gurobi.Var Y[6] (value 1.0)>,
 7: <gurobi.Var Y[7] (value 1.0)>}

In [16]:
Z

{(0, 0): <gurobi.Var Z[0,0] (value 18.0)>,
 (0, 1): <gurobi.Var Z[0,1] (value 0.0)>,
 (0, 2): <gurobi.Var Z[0,2] (value 0.0)>,
 (0, 3): <gurobi.Var Z[0,3] (value 0.0)>,
 (0, 4): <gurobi.Var Z[0,4] (value 0.0)>,
 (0, 5): <gurobi.Var Z[0,5] (value 0.0)>,
 (0, 6): <gurobi.Var Z[0,6] (value 0.0)>,
 (0, 7): <gurobi.Var Z[0,7] (value 0.0)>,
 (1, 0): <gurobi.Var Z[1,0] (value 0.0)>,
 (1, 1): <gurobi.Var Z[1,1] (value 18.0)>,
 (1, 2): <gurobi.Var Z[1,2] (value 0.0)>,
 (1, 3): <gurobi.Var Z[1,3] (value 0.0)>,
 (1, 4): <gurobi.Var Z[1,4] (value 0.0)>,
 (1, 5): <gurobi.Var Z[1,5] (value 0.0)>,
 (1, 6): <gurobi.Var Z[1,6] (value 0.0)>,
 (1, 7): <gurobi.Var Z[1,7] (value 0.0)>,
 (2, 0): <gurobi.Var Z[2,0] (value 0.0)>,
 (2, 1): <gurobi.Var Z[2,1] (value 0.0)>,
 (2, 2): <gurobi.Var Z[2,2] (value 13.0)>,
 (2, 3): <gurobi.Var Z[2,3] (value 0.0)>,
 (2, 4): <gurobi.Var Z[2,4] (value 0.0)>,
 (2, 5): <gurobi.Var Z[2,5] (value 0.0)>,
 (2, 6): <gurobi.Var Z[2,6] (value 0.0)>,
 (2, 7): <gurobi.Var Z[2,7] (va

In [17]:
# execution time
if __name__ == '__main__':
    print("Optimize time: %.5g sec" % csilsp_fal.Runtime)

Optimize time: 0.056175 sec


In [18]:
# Write results to file

results = "----------------------------------------------------------------" + "\n"
results += " Parameter T = %d " % (T) + "\n"
results += " Parameters d,h,p,s,C random generation from %d to %d " % (l_b,u_b) + "\n"
results += " Execution time = %.5g sec " % csilsp_fal.Runtime + "\n"
results += "----------------------------------------------------------------" + "\n\n\n"

In [19]:
with open("csilsp_fal_results.txt", "a") as write_file:
    write_file.write(results)